In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import load_boston, load_iris

from sklearn.linear_model import Ridge,Lasso,ElasticNet, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV # 최적의 파라미터를 찾을 수 있게 해줌
from sklearn.model_selection import cross_val_score

import mglearn # 알고리즘을 도식화해서 보여줌
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter('ignore')

In [ ]:
train_df=pd.read_excel('datas/hyundaiCar.xlsx',sheet_name='train') # sheet_name에는 인덱스 번호를 넣어줘도 가능 
test_df=pd.read_excel('datas/hyundaiCar.xlsx',sheet_name='test')   # 이 파일 같은 경우는 0 : train, 1 : test

In [ ]:
train_df

,가격,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,1885,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2190,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,1135,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,1645,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,1960,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동
...,...,...,...,...,...,...,...,...,...,...,...
66,3802,2015,중형,8.5,290,34.8,가솔린,0,3342,1901,자동
67,1270,2012,소형,13.3,108,13.9,가솔린,0,1396,1040,자동
68,2430,2015,준중형,12.8,186,41.0,디젤,0,1995,1665,자동
69,2870,2015,중형,17.7,156,19.3,가솔린,1,1999,1585,자동


In [ ]:
x_train=train_df.iloc[:,1:]
y_train=train_df[['가격']]

x_test=test_df.iloc[:,1:]
y_test=test_df[['가격']]

---

# 1.문자열 encoding


## Label Encoding

숫자의 크고 작음 때문에 나중에 비중에 문제가 생길 가능성이 높음

그래서 one hot encoding을 사용하는데 이건 메모리를 많이 씀

In [ ]:
lbl = LabelEncoder()
x_trainLabel = lbl.fit_transform( x_train['종류'])
x_trainLabel

array([2, 2, 1, 1, 0, 3, 3, 1, 3, 1, 2, 3, 2, 0, 1, 0, 0, 0, 3, 0, 0, 3,
       2, 0, 3, 3, 3, 1, 1, 2, 0, 0, 0, 0, 1, 0, 2, 0, 2, 1, 1, 1, 0, 0,
       0, 2, 1, 0, 2, 1, 3, 3, 0, 2, 2, 0, 3, 2, 0, 0, 2, 0, 1, 0, 0, 1,
       3, 1, 2, 3, 0])

In [ ]:
lbl.classes_

array(['대형', '소형', '준중형', '중형'], dtype=object)

In [ ]:
lbl.transform(['소형'])

array([1], dtype=int64)

## One Hot Encoding

0과 1로 구성

In [ ]:
oneh=OneHotEncoder()
x_trainone=oneh.fit_transform(x_train[['종류']]) # 이거는 matrix로 줘야함
x_trainone 

<71x4 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [ ]:
oneh.categories_

[array(['대형', '소형', '준중형', '중형'], dtype=object)]

## pandas get_dummies

In [ ]:
pd.get_dummies(x_train['종류'])

,대형,소형,준중형,중형
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0
...,...,...,...,...
66,0,0,0,1
67,0,1,0,0
68,0,0,1,0
69,0,0,0,1


In [ ]:
pd.get_dummies(x_train)
# 문자열 데이터를 전부 바꿔줌

,년식,연비,마력,토크,하이브리드,배기량,중량,종류_대형,종류_소형,종류_준중형,종류_중형,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,11.8,172,21.0,0,1999,1300,0,0,1,0,0,1,0,0,1
1,2015,12.3,204,27.0,0,1591,1300,0,0,1,0,0,1,0,0,1
2,2015,15.0,100,13.6,0,1368,1035,0,1,0,0,0,1,0,1,0
3,2014,14.0,140,17.0,0,1591,1090,0,1,0,0,0,1,0,0,1
4,2015,9.6,175,46.0,0,2497,1990,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2015,8.5,290,34.8,0,3342,1901,0,0,0,1,0,1,0,0,1
67,2012,13.3,108,13.9,0,1396,1040,0,1,0,0,0,1,0,0,1
68,2015,12.8,186,41.0,0,1995,1665,0,0,1,0,0,0,1,0,1
69,2015,17.7,156,19.3,1,1999,1585,0,0,0,1,0,1,0,0,1


In [ ]:
pd.get_dummies(x_train, columns=['연료','변속기'])

,년식,종류,연비,마력,토크,하이브리드,배기량,중량,연료_LPG,연료_가솔린,연료_디젤,변속기_수동,변속기_자동
0,2015,준중형,11.8,172,21.0,0,1999,1300,0,1,0,0,1
1,2015,준중형,12.3,204,27.0,0,1591,1300,0,1,0,0,1
2,2015,소형,15.0,100,13.6,0,1368,1035,0,1,0,1,0
3,2014,소형,14.0,140,17.0,0,1591,1090,0,1,0,0,1
4,2015,대형,9.6,175,46.0,0,2497,1990,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2015,중형,8.5,290,34.8,0,3342,1901,0,1,0,0,1
67,2012,소형,13.3,108,13.9,0,1396,1040,0,1,0,0,1
68,2015,준중형,12.8,186,41.0,0,1995,1665,0,0,1,0,1
69,2015,중형,17.7,156,19.3,1,1999,1585,0,1,0,0,1


In [ ]:
# Label endocing를 하고 싶다 하면 replace 사용
x_train['종류'].replace( ['대형','중형','준중형','소형'], [0,1,2,3] )

0     2
1     2
2     3
3     3
4     0
     ..
66    1
67    3
68    2
69    1
70    0
Name: 종류, Length: 71, dtype: int64

## 연습문제

라벨인코딩을 이용하여
2015, 준중형, 12.3, 204, 27,가솔린,0,1591,1300,자동
자동차가격을 예측하시요

팀별로 같이하세요

In [ ]:
lbl_1 = LabelEncoder()
x_train['종류'] = lbl_1.fit_transform(x_train['종류'])
lbl_2 = LabelEncoder()
x_train['연료'] = lbl_2.fit_transform(x_train['연료'])
lbl_3 = LabelEncoder()
x_train['변속기'] = lbl_3.fit_transform(x_train['변속기'])

# x_train.apply(LabelEncoder().fit_transform) 이렇게 하면 위의 과정처럼 번거롭게 하지 않아도 된다.



print(lbl_1.classes_)
print(lbl_2.classes_)
print(lbl_3.classes_)

model = Pipeline([('scl',StandardScaler()), ('clf',ElasticNet())])
param_value = {'clf__alpha' : [0.001,0.01,1,2,3,4], 'clf__l1_ratio':[0,0.1,0.3,0.5,0.7,0.9,1]}
gridSearch = GridSearchCV(model, param_grid=param_value, cv=10, verbose=1, scoring='r2')
gridSearch.fit(x_train,y_train)

gridSearch.best_estimator_.predict([[2015,2,12.3,204,27,1,0,1591,1300,1]])

print(gridSearch.best_params_)
print(gridSearch.best_estimator_)
print(gridSearch.best_estimator_.predict([[2015,2,12.3,204,27,1,0,1591,1300,1]]))

['대형' '소형' '준중형' '중형']
['LPG' '가솔린' '디젤']
['수동' '자동']
Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'clf__alpha': 1, 'clf__l1_ratio': 0.9}
Pipeline(steps=[('scl', StandardScaler()),
                ('clf', ElasticNet(alpha=1, l1_ratio=0.9))])
[2350.01358939]


[Parallel(n_jobs=1)]: Done 420 out of 420 | elapsed:    2.6s finished


## make_column_transformer

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
# 종류, 연료, 변속기 => one hot Encodinng으로 적용
myt = make_column_transformer(  ( OneHotEncoder(), ['종류','연료','변속기'] ), remainder = 'passthrough' )
result = myt.fit_transform( x_train)
result.shape

(71, 9)

In [ ]:
# model = Pipeline([('myt',myt),('scl',StandardScaler()), ('clf',Ridge())])
# param_value = {'clf__alpha' : [0.001,0.01,1,2,3,4]}
# gridSearch = GridSearchCV(model, param_grid=param_value, cv=10, verbose=1, scoring='r2')
# gridSearch.fit(x_train,y_train)


#  Ridge
model = make_pipeline(myt, StandardScaler(), Ridge())
param_value = {'ridge__alpha' : [0.001,0.1,1,2,3]} # Pipline과 다른 점은 따로 이름을 설정 안해줘도 된다 
gridSearch = GridSearchCV(model, param_grid=param_value, cv=5, verbose=1, scoring='r2')
gridSearch.fit(x_train,y_train)

display(gridSearch.best_estimator_)
display(gridSearch.best_score_)
display(gridSearch.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['종류', '연료', '변속기'])])),
                ('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=3))])

0.7227470861550487

{'ridge__alpha': 3}

In [ ]:
train_df=pd.read_excel('datas/hyundaiCar.xlsx',sheet_name='train')
test_df=pd.read_excel('datas/hyundaiCar.xlsx',sheet_name='test') 


x_train=train_df.iloc[:,1:]
y_train=train_df[['가격']]

x_test=test_df.iloc[:,1:]
y_test=test_df[['가격']]





In [ ]:
X_train = wth[:] 
y_train = wth[['기온']]

X_train

In [ ]:
# ElasticNet
model = make_pipeline(myt, StandardScaler(), ElasticNet())
param_value = {'elasticnet__alpha' : [0.001,0.1,1,2,3,4],'elasticnet__l1_ratio':[0.1,0.3,0.5,0.7,0.9]} 
gridS = GridSearchCV(model, param_grid=param_value,scoring='r2')
gridS.fit(x_train,y_train)

display(gridS.best_estimator_)
display(gridS.best_score_)
display(gridS.best_params_)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['종류', '연료', '변속기'])])),
                ('standardscaler', StandardScaler()),
                ('elasticnet', ElasticNet(alpha=1, l1_ratio=0.9))])

0.735820261410916

{'elasticnet__alpha': 1, 'elasticnet__l1_ratio': 0.9}

In [ ]:
gridS.best_estimator_.predict( x_test )

array([1803.11513233, 1189.31905429, 2741.08248165, 2432.55376989,
       2039.16328731, 2039.92440375, 2307.67541109, 1613.89388727,
       2475.62630444, 2522.81336211, 2330.1588652 , 5055.85183257,
       2042.43956228, 1539.96506713, 4641.8029698 , 1847.24976428,
       2336.442209  , 1157.45688564, 1146.56991696, 2005.38446591,
        944.5703928 , 1253.15262344, 7978.93482074, 2259.54993025,
       2758.86151709, 1583.27936545, 5728.46541879, 2297.24387513,
       2588.86989353,  999.30451114, 2910.5730406 ])

In [ ]:
xd = np.array( [[2015, '준중형', 12.3, 204, 27,'가솔린',0,1591,1300,'자동']])
xdataF = pd.DataFrame( xd, columns=x_test.columns)
xdataF
gridS.best_estimator_.predict( xdataF )

array([2261.86179938])

In [ ]:
myt = make_column_transformer(  ( OneHotEncoder(), ['종류','연료','변속기'] ), remainder = 'passthrough' )
result = myt.fit_transform( x_train)
result

model = make_pipeline(myt, PolynomialFeatures(degree = 2),StandardScaler(), ElasticNet())
param_value = {'elasticnet__alpha' : [0.001,0.1,1,2,3,4],'elasticnet__l1_ratio':[0.1,0.3,0.5,0.7,0.9]} 
gridS = GridSearchCV(model, param_grid=param_value,scoring='r2')
gridS.fit(x_train,y_train)

xd = np.array( [[2015, '준중형', 12.3, 204, 27,'가솔린',0,1591,1300,'자동']])
xdataF = pd.DataFrame( xd, columns=x_test.columns)
xdataF
gridS.best_estimator_.predict( xdataF )

array([2142.28182778])

---